# CSMCC16 Flight Coursework

## Set up

### Import packages

In [61]:
import pandas as pd
import datetime
from difflib import SequenceMatcher
import PySimpleGUI as sg
import math

In [62]:
# to view all 500 records, not top 5 and bottom 5
pd.set_option("display.max_rows", None, "display.max_columns", None)

### GUI settings

In [63]:
sg.SetOptions(
    background_color='#152238',
       text_element_background_color='#152238',
       element_background_color='#152238',
       text_color='#FFFFFF',
       scrollbar_color=None,
       input_elements_background_color='#FFFFFF',
       progress_meter_color = ('green', 'blue')
#        button_color=('white','#475841')
    )

True

## Create Functions

### Additional Columns

In [64]:
#calculate datetime values from epoch and arrival time
def calc_arrive_time(df):
    df['Arrive_epoch'] = pass_data['Depart_time_epoch']+df['Flight_time_min']*60
    df['Depart_time_datetime'] = pd.to_datetime(df['Depart_time_epoch'],unit='s')
    df['Arrive_time_datetime'] = pd.to_datetime(df['Arrive_epoch'],unit='s')
    df['Depart_time']=df['Depart_time_datetime'].dt.strftime("%H:%M:%S")
    df['Arrive_time']=df['Arrive_time_datetime'].dt.strftime("%H:%M:%S")


In [65]:
#create route (to use as unique id)
def route(df):
    df['Route'] = df['From_airport_IATA']+'-' + df['Dest_airport_IATA']+'-' + df['Depart_time']

In [66]:
#break route for details
def break_route(df):
    df[['From_airport_IATA','Dest_airport_IATA','Depart_time']]=df['Route'].str.split('-',expand=True)

### Import data

In [67]:
#import csv as dataframe
def import_dfs(pass_df,airport_df):
    global pass_data
    global airport_lat_lon
    global pass_data_slim
    global flights
    global freq_fliers
    pass_data = pd.DataFrame(pd.read_csv(pass_df,header=None))
    airport_lat_lon = pd.DataFrame(pd.read_csv(airport_df,header=None))
    # Add Column Names
    pass_data.columns = ['Passenger_id'
                            ,'Flight_id'
                            ,'From_airport_IATA'
                            ,'Dest_airport_IATA'
                            ,'Depart_time_epoch'
                            ,'Flight_time_min']
    airport_lat_lon.columns = ['Airport_name'
                            ,'Airport_IATA_code'
                            ,'Latitude'
                            ,'Longitude']

    ### Remove missing values
    pass_data=pass_data.dropna(subset=['Flight_id','Passenger_id'])
    #Add flight time info 
    calc_arrive_time(pass_data)
    #add route info
    route(pass_data)
    #select flights only
    flight_info = ['Flight_id'
                          ,'From_airport_IATA'
                          ,'Dest_airport_IATA'
                          ,'Route'         
                            ]
    flights = pass_data[flight_info]
    flights = flights.groupby(flight_info).count()
    flights.reset_index(inplace=True)
    #select required passenger only    
    pass_data_slim = pass_data[['Passenger_id','Flight_id','Route']]
    freq_fliers = flight_per_pass(pass_data_slim)


### Cleansing

In [68]:
# Remove any rows that do not fit good_format
def remove_bad_format(df,column,good_format):
    df =df.loc[df[column].str.contains(good_format)==True]
#add reset index?
    return df

In [69]:
# Remove row if not in a given list
def remove_list(df,column,list1):
    df = df[df[column].isin(list1)]
#     df.reset_index(inplace=True, drop = True)
    return df   

In [70]:
# Calculate number of passengers per flights
def pass_p_flight(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Flight_id').count()
#     df_new.reset_index(inplace=True, drop = True)
    return df_new 

In [71]:
# Calculate number of flights per passenger
def flight_per_pass(df):
    df2 = df[['Flight_id','Passenger_id']]
    df_new = df2.groupby('Passenger_id').count()
    df_new.reset_index(inplace=True)
    return df_new 

In [72]:
# Remove row if there is only 1
def remove_if_1(df,column):
    df = df.loc[df[column]!=1]
#     df.reset_index(inplace=True)  -- need this removed for passengers!!
    return df

### Error correction

In [73]:
# Remove flights where airport is not in airport list
def airline_check():
    global flights
    flights = remove_list(flights
                      ,'From_airport_IATA'
                      ,airport_lat_lon['Airport_IATA_code'
                                      ])
    flights = remove_list(flights
                      ,'Dest_airport_IATA'
                      ,airport_lat_lon['Airport_IATA_code'
                                      ])
    flights.reset_index(inplace=True, drop = True)    

In [74]:
#Remove flights that do not fit the flight id format
def flightid_check():
    global flights
    format_flightid = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z]"
    flights=remove_bad_format(flights,'Flight_id',format_flightid)

In [75]:
#remove poor format passenger id
def pass_id_check():
    global freq_fliers
    format_pass_id = r"[A-Z][A-Z][A-Z][0-9][0-9][0-9][0-9][A-Z][A-Z][0-9]"
    freq_fliers=remove_bad_format(freq_fliers,'Passenger_id',format_pass_id)

In [76]:
# Remove flights that have only had one passenger
def oneflight_check():
    global flights
    # Calculate flights per passenger
    flights_pass = pass_p_flight(pass_data)
    # Remove if just 1 flight
    flights_pp = remove_if_1(flights_pass,'Passenger_id')
    flights_pp.reset_index(inplace=True)  # need to check if required!  
    # Only keep flights in the flight pp list
    flights = remove_list(flights,'Flight_id',flights_pp['Flight_id'])

In [77]:
# update flight info
def update_flight_err (df,column1,column2):
    for ind in df.index:
        if (df[column1][ind] in flights[column1].to_list()) == False:
            #  if column 1 is wrong, use column 2
            use_column2 = df[column2][ind]
            #  get the row number of correct value
            row = flights[flights[column2] == use_column2].index[0]
            #  take the correct value from flights table
            df.loc[ind,column1]=flights[column1][row]
    return df

In [78]:
# Update passenger id bsed on a checklist
def update_pass_id(df,check_list):
        true = check_list.to_list()
        for ind in pass_data_slim.index:
            f = df['Passenger_id'][ind]
            # if the vlaue is not in the checklist...
            if (f in true)==False:
                test_best_true = []
                final_true = []
                # Check against all true values
                for t in true:
                    # SequesnceMatch Ratio will provide a similarity score
                        test_best_true.append([t,round(SequenceMatcher(None, t, f).ratio(),3)])
                test_best_true = pd.DataFrame(test_best_true)
                # Replace the false value with the true value that has highest score
                df.loc[ind,'Passenger_id']=test_best_true.iloc[test_best_true[1].idxmax(), 0]
        return df

## Objectives

## Objective 1: 
### Determine the number of flights from each airport; include a list of any airports not used.

In [79]:
def MapReduceObj1():
    global pass_data_slim
    global mapper
    global reducer
#     from_list = pass_data_slim['From_airport_IATA'].to_list()
    #mapper, where key = airport and value = 1 for each airport in from_list
    key2val2 = [[a, 1] for a in pass_data_slim['From_airport_IATA']]
    mapper = pd.DataFrame(key2val2, columns=['Airport_IATA_code','value'])
    #mapper

    #reducer - takes key and val from mapper to provide final key and value
    reducer=mapper.groupby(['Airport_IATA_code']).sum()
    reducer
    reducer.reset_index(inplace=True)

In [80]:
def format_obj1():
    global num_flights
    # merge full list of airports with number of flights from each airport
    airport_list = pd.DataFrame(airport_lat_lon['Airport_IATA_code'])
    # airport_list=airport_list)
    num_flights = airport_list.merge(reducer, how='outer', on=['Airport_IATA_code'])
    # clean up results
    num_flights=num_flights.fillna(0)
    num_flights=num_flights.sort_values(by=['value','Airport_IATA_code'], ascending=[False,True])
    # not sure whether to do the one below.... i think it looks neater....
    num_flights.reset_index(inplace=True, drop = True)
#     num_flights

## Objective 2
### Calculate the number of passengers on each flight.

In [81]:
def MapReduceObj2():
    global num_passengers
    global mapper
    global reducer
    #mapper, where key = airport and value = 1 for each airport in from_list
    key2val2 = [[a, 1] for a in pass_data_slim['Flight_id']]
    mapper = pd.DataFrame(key2val2, columns=['Flight_id','value'])
    #mapper

    #reducer - takes key and val from mapper to provide final key and value
    num_passengers=mapper.groupby(['Flight_id']).sum()
    #num_passengers
    #reducer
    num_passengers.reset_index(inplace=True)

In [82]:
def format_obj2():
    global num_passengers
    # clean up results
    #num_passengers=num_flights.fillna(0)
    num_passengers=num_passengers.sort_values(by=['value','Flight_id'], ascending=[False,True])
    # num_passengers.set_index('Flight_id', inplace=True, drop=True)
    num_passengers.reset_index(inplace=True, drop = True)


## Objective 3
### Output should include the passenger Id, relevant IATA/FAA codes, the departure time, the arrival time (times to be converted to HH:MM:SS format), and the flight times.

In [83]:
# Add additional flight information to flight id
def add_extra():
    global flight_extra_info
    global num_passengers
    global num_pas_extra
    global flights_min
    flights_min = flights[['Flight_id','Route']]
    flight_extra_info = ['Flight_id','Arrive_time', 'Flight_time_min','Route']
    flight_extra = pass_data[flight_extra_info]
    flight_extra = flight_extra.groupby(flight_extra_info).count()
    flight_extra.reset_index(inplace=True)
    flight_extra=flights_min.merge(flight_extra, how='inner', on=['Flight_id','Route'])
    num_pas_extra = num_passengers.merge(flight_extra, how='inner', on=['Flight_id'])

## Objective 4
### Calculate the line-of-sight (nautical) miles for each flight and the total travelled by each passenger and thus output the passenger having earned the highest air miles.

In [86]:
# pass_data_slim

In [87]:
# Calulate miles per flight,

def calculate_miles(df_in):
    global miles_per_flight
    miles_per_flight = []
    for ind in df_in.index:
            #Take from airport
            p= df_in['Passenger_id'][ind]
            x= df_in['From_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == x].index[0])
            #Give me the latitude and longitude
            a = airport_lat_lon['Latitude'][row]
            b = airport_lat_lon['Longitude'][row]
            #Take dest airport
            y = df_in['Dest_airport_IATA'][ind]
            #Find it in airport lat long list
            row = (airport_lat_lon[airport_lat_lon['Airport_IATA_code'] == y].index[0])
            #Give me the latitude and longitude
            c = airport_lat_lon['Latitude'][row]
            d = airport_lat_lon['Longitude'][row]
            miles = math.sqrt((c-a)**2+(d-b)**2)
            miles_per_flight.append([df_in['Passenger_id'][ind],miles])
    miles_per_flight= pd.DataFrame(miles_per_flight
                                   , columns=['Passenger_id','Miles']
                                  )    


In [88]:
def MapReduceObj4():
    global miles_per_pass
    # mapper
    key2val2 = pd.DataFrame()
    for index,row in miles_per_flight.iterrows():
        key2val2=key2val2.append(miles_per_flight.loc[index])
        mapper = pd.DataFrame(key2val2, columns=['Miles','Passenger_id'])
    #reducer - takes key and val from mapper to provide final key and value
    miles_per_pass=mapper.groupby(['Passenger_id']).sum().round(1)
    #reducer
    miles_per_pass.reset_index(inplace=True)

In [89]:
def format_obj4():
    global miles_per_pass
    # clean up results
    #num_passengers=num_flights.fillna(0)
    miles_per_pass=miles_per_pass.sort_values(by=['Miles'], ascending=[False])
    miles_per_pass.set_index('Passenger_id', inplace=True, drop=True)
    miles_per_pass.reset_index(inplace=True, drop = False)


## GUI

### Create windows

In [90]:
#Welcome Window
def welcome_win():
    
    layout = [
                [sg.Text("University of Reading Flight Tracker", size=(30, 1), font=('Tahoma', 20), key='Text')]
               , [sg.Text("This prototype demonstrates how a MapReduce type algorithm can be used to analyse flights. Please press a button to continue"
                          , size=(30, 5), font=('Tahoma', 18), key='Text2')]
              , [sg.Button("Close",font=(14)), sg.Button("Import Data",font=(14), key='-import-')]]

    # Create the window
    window = sg.Window("Welcome", layout, margins=(10 #width
                                                , 10 #height
                                               ))

    # Create an event loop
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == "Close" or event == sg.WIN_CLOSED:
            break
        elif event == '-import-':
    #         pass_data = pd.DataFrame(pd.read_csv('AComp_Passenger_data.csv',header=None))    
            import_dfs('AComp_Passenger_data.csv','Top30_airports_LatLong.csv')
            print('pass_data data imported')
    #         airport_lat_lon = pd.DataFrame(pd.read_csv('Top30_airports_LatLong.csv',header=None))
            print('airport data imported')
            data_loaded_win()

    window.close()

In [91]:
#Data Loaded Window
def data_loaded_win():

    layout = [
#                 [sg.Text("Data Loaded", size=(30, 1), font=('Tahoma', 20), key='Text')]
                [sg.Text("The data has been loaded"
                          , size=(45, 1), font=('Tahoma', 18), key='Text2')]
               , [sg.Text("Go to error correction if you have not yet done so."
                          , size=(45, 1), font=('Tahoma', 18), key='Text3')]
               , [sg.Text("Go to objectives if you have already corrected errors."
                          , size=(45, 1), font=('Tahoma', 18), key='Text4')]
               , [sg.Text("Or if your data has no errors, go to objectives."
                          , size=(45, 1), font=('Tahoma', 18), key='Text5')]
              , [sg.Button("Error Correction",font=(14),key='-error-'), sg.Button("Objectives",font=(14), key='-obj-')]]

    # Create the window
    window = sg.Window("Data Loaded", layout, margins=(10, 10))

    # Create an event loop
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == sg.WIN_CLOSED:
#         if event == '-obj-' or event == sg.WIN_CLOSED:
            break
        elif event == '-error-':
            error_corr_win()
            print('go to error correction')
        elif event == '-obj-':
            break_route(pass_data_slim)
            print('break route')
            objectives_win()
            print('objective window')
    window.close()

In [92]:
#Error Correction Window
def error_corr_win():

    layout = [
               [sg.Text("Click to detect and correct errors. Errors can be found in flight or passenger information"
                          , size=(30, 4), font=('Tahoma', 18), key='Text2')]
              , [sg.Button("Correct Flight Information",font=(14), size=(25, 1),key='-flight_err-')]
              , [sg.Button("Correct Passenger Information",font=(14), size=(25, 1),key='-pass_err-')]
              , [sg.Text("",font=(14),key='-space-')]
              , [sg.Button("Close",font=(14),key='-exit-')]]

    # Create the window
    window = sg.Window("Error Correction", layout, margins=(10, 10))

    # Create an event loop
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == '-exit-' or event == sg.WIN_CLOSED:
            break
        elif event == '-obj-':
            print('go to objectives')
        elif event == '-flight_err-':
            flight_err_win()
            print('go to correct flight errors')
        elif event == '-pass_err-':
            pass_err_win()
            print('go to correct passenger errors')

    window.close()

In [93]:
#Flight Correction Window
def flight_err_win():
    global flights
    global pass_data_slim

    layout = [
               [sg.Text("Use the checkboxes to mark which error correction methods are to be included."
                          , size=(30, 4), font=('Tahoma', 18), key='Text2')]
            ,[sg.Checkbox('Check airline against list', font=('Tahoma', 14), enable_events=True, key='-airline-')]
            ,[sg.Checkbox('Check flight id format', font=('Tahoma', 14), enable_events=True, key='-format-')]
            ,[sg.Checkbox('Check if only one flight', font=('Tahoma', 14), enable_events=True, key='-oneflight-')]
              , [sg.Text("",font=(14),key='-space-')]
              , [sg.Button("Back",font=(14),key='-exit-')]]

    # Create the window
    window = sg.Window("Error Correction", layout, margins=(10, 10))

    # Create an event loop
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == '-exit-' or event == sg.WIN_CLOSED:
            break
        elif event == '-airline-':
            airline_check()
            print('correct based on airline')
        elif event == '-format-':
            flightid_check()
            print('correct based on flight id format')
        elif event == '-oneflight-':
            oneflight_check()
            print('correct based on one flight')
    flights.reset_index(inplace=True, drop = True)
    # Correct the errors
    pass_data_slim = update_flight_err(pass_data_slim,'Route','Flight_id')
    pass_data_slim = update_flight_err(pass_data_slim,'Flight_id','Route')
#     print(pass_data_slim)
    window.close()

In [94]:
#Passenger Error Correction Window
def pass_err_win():
    global pass_data_slim
    global freq_fliers

    layout = [
               [sg.Text("Use the checkboxes to mark which error correction methods are to be included."
                          , size=(30, 4), font=('Tahoma', 18), key='Text2')]
            ,[sg.Checkbox('Check passenger id format', font=('Tahoma', 14), enable_events=True, key='-format-')]
            ,[sg.Checkbox('Check if only one flight', font=('Tahoma', 14), enable_events=True, key='-oneflight-')]
              , [sg.Text("",font=(14),key='-space-')]
              , [sg.Button("Back",font=(14),key='-exit-')]]

    # Create the window
    window = sg.Window("Error Correction", layout, margins=(10, 10))

    # Create an event loop
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == '-exit-' or event == sg.WIN_CLOSED:
            break
        elif event == '-format-':
            pass_id_check()
            print('correct based on passenger id format')
        elif event == '-oneflight-':
            freq_fliers = remove_if_1(freq_fliers,'Flight_id')
            freq_fliers.reset_index(inplace=True, drop = True)
            print('correct based on one flight')
    freq_fliers.reset_index(inplace=True, drop = True)
    #Correct the errors
    pass_data_slim = update_pass_id(pass_data_slim,freq_fliers['Passenger_id'])
#     print(pass_data_slim)
    window.close()

In [95]:
#Objectives Window
def objectives_win():

    layout = [
               [sg.Text("Click to view each objective"
                          , size=(30, 2), font=('Tahoma', 18), key='Text2')]
              , [sg.Button("Number of flights",font=(14), size=(30, 1),key='-obj1-')]
              , [sg.Button("Number of passengers",font=(14), size=(30, 1),key='-obj2-')]
              , [sg.Button("Flight Miles",font=(14), size=(30, 1),key='-obj4-')]
              , [sg.Text("",font=(14),key='-space-')]
              , [sg.Button("Exit",font=(14),key='-exit-')]]

    # Create the window
    window = sg.Window("Objectives", layout, margins=(10, 10))

    # Create an event loop
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == '-exit-' or event == sg.WIN_CLOSED:
            break
        elif event == '-obj1-':
            print('obj1')
            MapReduceObj1()
            format_obj1()
            obj_win(num_flights,'Number of Flights from given airport',['Airport','Number of flights'])
        elif event == '-obj2-':
            MapReduceObj2()
            format_obj2()
            obj_win2(num_passengers,'Number of passengers per flight',['  Flight id  ','Number of passengers'])
#             pass_err_win()
            print('obj2')
        elif event == '-obj4-':
            print('obj4')
            calculate_miles(pass_data_slim)
            MapReduceObj4()
            format_obj4()
            obj_win(miles_per_pass,'Number of miles per passenger',['Passenger id','Number of miles'])

    window.close()

In [96]:
def obj_win (filename,title,header_list):
    #Create table
    sg.set_options(auto_size_buttons=True)
    df = filename
#     keep_on_top = True

    data = []
#     header_list = []
    data = df.values.tolist()               # read everything else into a list of rows
#     header_list = ['column' + str(x) for x in range(len(data[0]))]
    layout = [
        [sg.Text(title, size=(30, 1), font=('Tahoma', 20), key='Text')],
        [sg.Table(values=data,
#                   size=(60,10),
                  headings=header_list,
                  font=('Tahoma', 20),
                  display_row_numbers=False,
                  auto_size_columns=True,
                  num_rows=min(10, len(data)))],
        [sg.Button('Close',font=('Tahoma', 20),key='-close-')]      

    ]

    window = sg.Window('Table', layout, grab_anywhere=False)
#     event, values = window.read()
    
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == '-close-' or event == sg.WIN_CLOSED:
            break
    window.close()

# table_example('Top30_airports_LatLong.csv')

In [97]:
def obj_win2 (filename,title,header_list):
    #Create table
    sg.set_options(auto_size_buttons=True)
    df = filename
#     keep_on_top = True

    data = []
#     header_list = []
    data = df.values.tolist()               # read everything else into a list of rows
#     header_list = ['column' + str(x) for x in range(len(data[0]))]
    layout = [
        [sg.Text(title, size=(30, 1), font=('Tahoma', 20), key='Text')],
        [sg.Table(values=data,
#                   size=(60,10),
                  headings=header_list,
                  font=('Tahoma', 20),
                  display_row_numbers=False,
                  auto_size_columns=True,
                  num_rows=min(10, len(data)))],
        [sg.Button('Close',font=('Tahoma', 20),key='-close-'),sg.Button('Add extra info',font=('Tahoma', 20),key='-extra-')]      

    ]

    window = sg.Window('Table', layout, grab_anywhere=False)
#     event, values = window.read()
    
    while True:
        event, values = window.read()
        # End program if user closes window or
        # presses the OK button
        if event == '-close-' or event == sg.WIN_CLOSED:
            break
        if event == '-extra-':
            extra_headers=['Flight_id','Count Passengers','Route','Arrival time','Flight mins']
            obj_win (num_pas_extra,'Number of passengers',extra_headers)
            print('-extra-')
#             add_extra()
    
            
    window.close()

# table_example('Top30_airports_LatLong.csv')

### Start Application

In [98]:
welcome_win()

pass_data data imported
airport data imported
correct based on airline
correct based on flight id format
correct based on one flight


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


go to correct flight errors
correct based on passenger id format
correct based on one flight


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


go to correct passenger errors
go to error correction
break route
obj1
-extra-
obj2
obj4
objective window
